## Sentinel-2 Shoreline Extraction over Morecambe Bay
 Author : Olivia Brennan \
 Institution: University College London

### 1. Preliminary setup:

In [1]:
#import packages:

import datetime
from datetime import datetime, timedelta
import pandas as pd
import re
import numpy as np
from dateutil.parser import parse
import ee
import folium
from folium import plugins
import numpy as np

In [2]:
#initialize Google Earth Engine project:

ee.Authenticate() # account is authenticated
ee.Initialize(project='x-avenue-463113-j8') # project ID for 'Olivia's Dissertation project'

In [3]:
#define the case study boundary: Morecambe Bay
morecambe_boundary = ee.Geometry.Rectangle([-3.3, 53.90, -2.70, 54.25])

### 2. Extract Sentinel-2 data from GEE:
Sentinel-2 data is availible from 2017 to the end of April 2025.\
Define the start and end time for data.\
Call scenes from GEE for each year.\
Export to Google Drive.

Once this has been run and data moved from google drive to local file, this section need not be ran again.


In [4]:
# loop through years 2017 to 2025 to define start and end data of datasets
# 2025 data ends in May so end date is different

years = list(range(17, 26))
for year in years:
    
    # define start and end dates for each year:
    if year == 25:
        start_date = f"20{year}-01-01"
        end_date = f"20{year}-05-01"
    else:
        start_date = f"20{year}-01-01"
        end_date = f"20{year+1}-01-01"
        
    
    # Filter Sentinel-2 collection based on start and end dates
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(morecambe_boundary) \
        .filterDate(start_date, end_date) \
        .map(lambda img: ee.Feature(None, {
            'scene_id': img.get('system:index'),
            'timestamp': img.get('system:time_start'),
            'cloudiness': img.get('CLOUDY_PIXEL_PERCENTAGE')
        }))
    
    # Convert to a FeatureCollection
    features = ee.FeatureCollection(collection)
    #define filename
    filename = (f"Morecambe_Metadata_{start_date}_{end_date}")
    # Export as a CSV to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=features,
        folder='GEE_exports',
        description= filename,
        fileFormat='CSV'
    )   
    #task.start() #uncomment to run tasks
    print(f'Successfully exported file: {filename} to Google Drive')

print("Don't forget to move the exported metadata from Google Drive to a local folder!" )

Successfully exported file: Morecambe_Metadata_2017-01-01_2018-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2018-01-01_2019-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2019-01-01_2020-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2020-01-01_2021-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2021-01-01_2022-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2022-01-01_2023-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2023-01-01_2024-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2024-01-01_2025-01-01 to Google Drive
Successfully exported file: Morecambe_Metadata_2025-01-01_2025-05-01 to Google Drive
Don't forget to move the exported metadata from Google Drive to a local folder!


### 3. Link Metadata to Tide gauge data in dictionary
Ensure tide gauge files are saved locally.\
Ensure metadata has been moved from Google Drive to local file.

In [5]:
#set up a dictionary to store the merged results for each year:
merged_data = {}

years = list(range(17, 26))
for year in years:
    
    # define start and end dates for each year:
    if year == 25:
        start_date = f"20{year}-01-01"
        end_date = f"20{year}-05-01"
    else:
        start_date = f"20{year}-01-01"
        end_date = f"20{year+1}-01-01"


#format the metadata for analysis:
    
    # Load the exported metadata for this year
    df = pd.read_csv(f"Morecambe_Metadata_{start_date}_{end_date}.csv")
    # Convert Unix epoch (milliseconds) to datetime
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    # Format to 'YYYY MM DD HH MI SS'
    df['formatted_datetime'] = df['datetime'].dt.strftime('%Y %m %d %H %M %S')
    # Save to new CSV - this file cannot be open in excel when running this 
    df.to_csv(f"Formatted_Morecambe_Metadata_{start_date}_{end_date}.csv", index=False)
    #name it 'metadata'
    metadata = pd.read_csv(f"Formatted_Morecambe_Metadata_{start_date}_{end_date}.csv")
    # round the metadata to nearest 15 mins
    metadata['datetime'] = pd.to_datetime(metadata['datetime'], errors='coerce')
    metadata['rounded_time'] = metadata['datetime'].dt.round('15min')
    #print(f'formatted metadatafile for year 20{year}')

#format the tide data:

    #load tide data for this year
    tides = pd.read_csv(f"HEY{year}_clean.csv", skiprows=12,header=None)
    tides = tides.iloc[:, [1, 2, 7]] #extract only relavent collumns
    tides.columns = ['Date', 'Time', 'Tide_Level'] #rename collumns 
    tides['Tide_Level'] = tides['Tide_Level'].astype(str).apply(lambda x: re.sub(r'[^\d.]', '', x)).str.strip()
    tides['Tide_Level'] = tides['Tide_Level'].replace('', np.nan)
    #ensure dates are formatted consistently:
    def safe_parse(row):
        try:
            return parse(f"{row['Date']} {row['Time']}", dayfirst=True)
        except Exception:
            return pd.NaT     
    #combine the data and time:
    tides['rounded_time'] = tides.apply(safe_parse, axis=1)
    #preview data:
    tides['Tide_Level'] = pd.to_numeric(tides['Tide_Level'], errors='coerce')
    count = 0
    for x in tides:
        try:
            if float(x) > 10:
                count += 1
        except ValueError:
            pass  # skip anything that can't be converted to float
    
    #print( f'formatted tide data for year 20{year}')

#combine the metadata to the tide data based on the timestamp
  
    #create a 'combined' dataframe
    combined_df = pd.merge(
        metadata,
        tides[['rounded_time', 'Tide_Level']],
        on='rounded_time',
        how='left'
    )

#add combined data to dictionary    
    merged_data[year] = combined_df
    
    print(f'Combined metadata with tide data for year 20{year}')



Combined metadata with tide data for year 2017
Combined metadata with tide data for year 2018
Combined metadata with tide data for year 2019
Combined metadata with tide data for year 2020
Combined metadata with tide data for year 2021
Combined metadata with tide data for year 2022
Combined metadata with tide data for year 2023
Combined metadata with tide data for year 2024
Combined metadata with tide data for year 2025


### 4. Filter by cloudiness, tide level and year
define how cloudy a scene can be, what tide state it should be in and for what year...

In [6]:
# for whichever year you wish to examine
year = 19

# what is the maximum percentage cloudiness a scene can be?
cloud_threshold = 40 

# what tide range to look within? (metres above chart datum)
tide_min = float('0')
tide_max = float('4')

#apply filter: 
filtered_by_year = merged_data[year]
filtered_by_cloudiness = filtered_by_year[filtered_by_year['cloudiness'] < cloud_threshold] 
filtered_by_tide = filtered_by_cloudiness[(filtered_by_cloudiness['Tide_Level'] < tide_max) \
                                        & (filtered_by_cloudiness['Tide_Level'] > tide_min) \
                                        & (filtered_by_cloudiness['Tide_Level'].notna())
                                        ]   

print(f'After filtering, the number of availible scenes is {filtered_by_tide.shape[0]}')

After filtering, the number of availible scenes is 31


In [7]:
#preview the data that has been filtered:
filtered_by_tide

,system:index,cloudiness,scene_id,timestamp,.geo,datetime,formatted_datetime,rounded_time,Tide_Level
40,20190128T113331_20190128T113557_T30UVE,23.972876,20190128T113331_20190128T113557_T30UVE,1.548675e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-01-28 11:36:18.000,2019 01 28 11 36 18,2019-01-28 11:30:00,2.244
41,20190128T113331_20190128T113557_T30UVF,1.877995,20190128T113331_20190128T113557_T30UVF,1.548675e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-01-28 11:36:06.000,2019 01 28 11 36 06,2019-01-28 11:30:00,2.244
42,20190128T113331_20190128T113557_T30UWE,0.877093,20190128T113331_20190128T113557_T30UWE,1.548675e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-01-28 11:36:13.000,2019 01 28 11 36 13,2019-01-28 11:30:00,2.244
43,20190128T113331_20190128T113557_T30UWF,2.870043,20190128T113331_20190128T113557_T30UWF,1.548675e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-01-28 11:36:04.000,2019 01 28 11 36 04,2019-01-28 11:30:00,2.244
45,20190130T112319_20190130T112319_T30UVF,39.833314,20190130T112319_20190130T112319_T30UVF,1.548848e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-01-30 11:26:10.000,2019 01 30 11 26 10,2019-01-30 11:30:00,3.728
47,20190130T112319_20190130T112319_T30UWF,37.087116,20190130T112319_20190130T112319_T30UWF,1.548848e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-01-30 11:26:06.000,2019 01 30 11 26 06,2019-01-30 11:30:00,3.728
88,20190227T113311_20190227T113314_T30UVE,3.204113,20190227T113311_20190227T113314_T30UVE,1.551267e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-02-27 11:36:17.000,2019 02 27 11 36 17,2019-02-27 11:30:00,2.870
89,20190227T113311_20190227T113314_T30UVF,3.749052,20190227T113311_20190227T113314_T30UVF,1.551267e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-02-27 11:36:03.000,2019 02 27 11 36 03,2019-02-27 11:30:00,2.870
90,20190227T113311_20190227T113314_T30UWE,0.253419,20190227T113311_20190227T113314_T30UWE,1.551267e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-02-27 11:36:12.000,2019 02 27 11 36 12,2019-02-27 11:30:00,2.870
91,20190227T113311_20190227T113314_T30UWF,0.107855,20190227T113311_20190227T113314_T30UWF,1.551267e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",2019-02-27 11:35:58.000,2019 02 27 11 35 58,2019-02-27 11:30:00,2.870


### 5. Visualise the filtered selection
Use the dropdown box to toggle each scene on and off.\
Make own judgement about useability of each scene 

In [8]:
#extract scene IDs 
scene_ids = filtered_by_tide['scene_id'].tolist()

#import geehydro  # Optional, adds EE layers to folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

#define mapping region
morecambe_boundary = ee.Geometry.Rectangle([-3.3, 53.90, -2.70, 54.25])


# Define map center 
map_center = [54.05, -2.9]
myMap = folium.Map(location=map_center, zoom_start=10)

# Add Sentinel-2 images to the map
for sid in scene_ids: 
    image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/' + sid)
    vis_params = {
        'bands': ['B4', 'B3', 'B2'],  # True color
        'min': 0,
        'max': 3000,
        'gamma': 1.3
    }
    image = image.clip(morecambe_boundary)  # Clip to study area
    myMap.add_ee_layer(image, vis_params, sid)

# Add layer control
myMap.add_child(folium.LayerControl())

# Display map
myMap

### 6. Extract Shorelines for approved scenes using NDWI thresholding
based on the date of scenes which looked good enough to use from the visualisation above^\
Apply Normalized difference index (NDWI) to pixels.\
Defined water as pixels where NDWI>0.\
Exported shapefiles can be found in Google Drive.

In [9]:
# Define the dates of the scenes we want to extract:
# example set of dates:
dates = ["2018-05-13",
         "2017-04-08",
         "2019-04-08",
         "2020-09-29",
         "2020-04-27",
         "2024-08-31"]

#Define Morecambe Bay ROI
morecambe_boundary = ee.Geometry.Rectangle([-3.3, 53.90, -2.70, 54.25])

#Create an function which finds NDWI based on spectral bands
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI') #norm difference B3 (green) and B8 (NIR)
    return image.addBands(ndwi)


In [10]:
#loop through each date and extract a shoreline polygon based on NDWI threshold
for date in dates:
    
    start_date = datetime.strptime(date,"%Y-%m-%d")
    end_date = start_date + timedelta(days=1)

    #call the set of scenes
    sentinel_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                       .filterDate(start_date, end_date) 
                       .filterBounds(morecambe_boundary)
                       .median())
    sentinel_image = sentinel_collection.clip(morecambe_boundary) #clip to ROI
    #apply NDWI filter
    sentinel_ndwi = add_ndwi(sentinel_image) 
    # Create the three masks
    ndwi_band = sentinel_ndwi.select('NDWI')
    water = ndwi_band.gt(0.0) #water is where NDWI is greater than 0
    land = ndwi_band.lte(0.0) #land is where NDWI is less than 0
    #just water
    water_mask = ndwi_band.gt(0).selfMask() 


    #extract shoreline to vectors
    shore_polygons = water_mask.reduceToVectors(
    geometry = morecambe_boundary,  
    scale = 10,           # Sentinel-2 resolution is 10m
    geometryType ='polygon',
    labelProperty ='class',
    maxPixels = 1e13
    )

    filedescription = f"waterlineExport{start_date}"

    #use GEE to export the vecotr to a shapefile
    #check task progress online
    task = ee.batch.Export.table.toDrive(
        collection=shore_polygons,
        description=filedescription,
        folder='GEE_exports',
        fileNamePrefix=filedescription,
        fileFormat='SHP'
    )
    task.start()
    print (f"shoreline export complete for date: {start_date}")

shoreline export complete for date: 2018-05-13 00:00:00
shoreline export complete for date: 2017-04-08 00:00:00
shoreline export complete for date: 2019-04-08 00:00:00
shoreline export complete for date: 2020-09-29 00:00:00
shoreline export complete for date: 2020-04-27 00:00:00
shoreline export complete for date: 2024-08-31 00:00:00


### 7. Calculate statistics for Sentinel-2 DATA
- how many scenes were captured over Morecambe Bay per year?
- how many scenes were at extreme tidal stages ?
- how many scenes were cloudy ?

In [11]:
#remove duplicate dates from dictionary (eg more than one scene on same day)
unique_dfs = {}

for year, df in merged_data.items():
    # Remove duplicates based on 'rounded_time', keep the first occurrence
    unique_df = df.drop_duplicates(subset='rounded_time', keep='first')

    # Store the cleaned DataFrame in the new dictionary
    unique_dfs[year] = unique_df

    #print(f"20{year}: {len(unique_df)} rows after removing duplicates")

In [12]:
#create a dataframe to store my stats
df_stats = pd.DataFrame({'Year': range(17,26)})

for year in range(17,26):
    #calculate stats:                                          # for each year:
    unique_count = unique_dfs[year]['rounded_time'].nunique()  #find the number of unique fly-overs
    maxtide = unique_dfs[year]['Tide_Level'].max()             #find the maximum tide level
    mintide = unique_dfs[year]['Tide_Level'].min()             #find the minimum tide level
    tides_above9 = (unique_dfs[year]['Tide_Level']>9).sum()    #find number of tides above 9m
    tides_above10 = (unique_dfs[year]['Tide_Level']>10).sum()  #find number of tides above 10m
    tides_below1 = (unique_dfs[year]['Tide_Level']<1).sum()    #find number of tides below 1m
    tides_below2 = (unique_dfs[year]['Tide_Level']<2).sum()    #find number of tides below 2m
    tide_range = maxtide - mintide                             #find the range of tides 
    av_cloud = unique_dfs[year]['cloudiness'].mean()           #find the average cloudiness score
    cloud_below40 = (unique_dfs[year]['cloudiness']<40).sum()  #find the number of scenes where cloudiness was below 40%
    cloud_below40and3m = ((unique_dfs[year]['cloudiness'] < 40) \
                          & (unique_dfs[year]['Tide_Level'] < 3)).sum()
    cloud_below40and1m = ((unique_dfs[year]['cloudiness'] < 40) \
                          & (unique_dfs[year]['Tide_Level'] < 1)).sum()
    cloud_below40and10m = ((unique_dfs[year]['cloudiness'] < 40) \
                          & (unique_dfs[year]['Tide_Level'] > 10)).sum()
    cloud_below40and9_5m = ((unique_dfs[year]['cloudiness'] < 40) \
                          & (unique_dfs[year]['Tide_Level'] > 9.5)).sum()
    
    #add stats to table:
    df_stats.loc[df_stats['Year'] == year, 'Fly-overs'] = int(unique_count) 
    df_stats.loc[df_stats['Year'] == year, 'Maximum observed tide (m)'] = maxtide
    df_stats.loc[df_stats['Year'] == year, 'Minimum observed tide (m)'] = mintide
    df_stats.loc[df_stats['Year'] == year, 'Observed tidal range (m)'] = tide_range
    df_stats.loc[df_stats['Year'] == year, 'Scenes where T>9m'] = tides_above9
    df_stats.loc[df_stats['Year'] == year, 'Scenes where T>10m'] = tides_above10
    df_stats.loc[df_stats['Year'] == year, 'Scenes where T<1m'] = tides_below1
    df_stats.loc[df_stats['Year'] == year, 'Scenes where T<2m'] = tides_below2
    df_stats.loc[df_stats['Year'] == year, 'Average Cloudiness (%)'] = (av_cloud)
    df_stats.loc[df_stats['Year'] == year, 'Scenes where cloud<40%'] = cloud_below40
    df_stats.loc[df_stats['Year'] == year, 'Cloud<40% & T<3m'] = cloud_below40and3m
    df_stats.loc[df_stats['Year'] == year, 'Cloud<40% & T<1m'] = cloud_below40and1m
    df_stats.loc[df_stats['Year'] == year, 'Cloud<40% & T>10m'] = cloud_below40and10m
    df_stats.loc[df_stats['Year'] == year, 'Cloud<40% & T>9.5m'] = cloud_below40and9_5m

#make the year 2000 and..
df_stats['Year'] = (df_stats['Year'] + 2000)

#formatting the table:
formatters = {
    'Fly-overs': '{:.0f}'.format,
    'Max tide': '{:.2f}'.format,
    'Min tide': '{:.2f}'.format,      
    'tidal range': '{:.2f}'.format,
    'T>9m': '{:.0f}'.format,
    'T>10m': '{:.0f}'.format,
    'T<1m': '{:.0f}'.format,      
    'T<2m': '{:.0f}'.format,
    'av cloud': '{:.2f}'.format,
    'Cloud<40%': '{:.0f}'.format,
    'Cloud<40% & T<3m': '{:.0f}'.format,      
    'Cloud<40% & T<1m': '{:.0f}'.format
}

#formatted_df = df_stats.to_string(formatters=formatters, index=False)
#print(formatted_df)

#pd_formatted_df = pd.DataFrame(formatted_df)


#display the original statistics table:
#df_stats
#pd_formatted_df



df_stats.style.format({
    'Fly-overs': '{:.0f}',
    'Maximum observed tide (m)': '{:.2f}',
    'Minimum observed tide (m)': '{:.2f}',
    'Observed tidal range (m)': '{:.2f}',
    'Scenes where T>9m': '{:.0f}',
    'Scenes where T>10m': '{:.0f}',
    'Scenes where T<1m': '{:.0f}',
    'Scenes where T<2m': '{:.0f}',
    'Average Cloudiness (%)': '{:.1f}',
    'Scenes where cloud<40%': '{:.0f}',
    'Cloud<40% & T<3m': '{:.0f}',
    'Cloud<40% & T<1m': '{:.0f}',
    'Cloud<40% & T>10m': '{:.0f}',
    'Cloud<40% & T>9.5m': '{:.0f}'
})

,Year,Fly-overs,Maximum observed tide (m),Minimum observed tide (m),Observed tidal range (m),Scenes where T>9m,Scenes where T>10m,Scenes where T<1m,Scenes where T<2m,Average Cloudiness (%),Scenes where cloud<40%,Cloud<40% & T<3m,Cloud<40% & T<1m,Cloud<40% & T>10m,Cloud<40% & T>9.5m
0,2017,57,9.75,2.10,7.65,9,0,0,0,52.4,22,2,0,0,2
1,2018,148,10.55,2.01,8.55,23,3,0,0,60.3,51,9,0,1,3
2,2019,145,10.51,0.08,10.43,23,3,2,4,66.7,37,6,1,0,1
3,2020,144,10.37,1.70,8.67,23,1,0,2,60.9,43,5,0,0,1
4,2021,146,10.02,2.12,7.90,19,1,0,0,63.4,40,5,0,1,4
5,2022,146,10.47,2.12,8.36,26,4,0,0,63.0,48,4,0,3,5
6,2023,146,10.43,0.09,10.35,25,4,3,3,68.4,29,1,0,0,2
7,2024,145,10.32,0.09,10.23,18,4,13,18,70.6,30,6,5,0,1
8,2025,51,10.28,2.52,7.75,9,2,0,0,68.8,13,1,0,0,1
